In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm


DATADIR = 'videos'
CATEGORIES = ['Fire', 'NoFire']

IMG_SIZE = 128
def create_training_data():
    training_data = []
    for category in CATEGORIES:  

        path = os.path.join(DATADIR,category) 
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=C 1=O

        for img in tqdm(os.listdir(path)):  # iterate over each image
            try:
                img_array = cv2.imread(os.path.join(path,img))  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
              
    return training_data


In [2]:
training_data = create_training_data()

  0%|          | 0/1124 [00:00<?, ?it/s]

100%|██████████| 1301/1301 [00:12<00:00, 101.21it/s]


In [3]:
import random

print(len(training_data))
random.shuffle(training_data)
for sample in training_data[:10]:
    print(sample[1])

2423
0
1
0
0
0
1
1
1
0
0


In [4]:
X = []
Y = []

for features,label in training_data:
    X.append(features)
    Y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
X = X/255.0
X.shape[1:]
Y = np.array(Y)

In [5]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D

model = Sequential()


model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=X.shape[1:]))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=128, activation='relu'))

model.add(Dense(units=2, activation = 'softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

2024-05-08 19:47:46.403573: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 19:47:46.403642: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 19:47:46.404977: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-08 19:47:46.510556: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-08 19:47:50.493605: I tensorflow/compiler/

In [6]:
history = model.fit(X, Y, batch_size=32, epochs=100,validation_split=0.3)

Epoch 1/100


2024-05-08 19:47:52.549948: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 333447168 exceeds 10% of free system memory.
2024-05-08 19:47:52.742965: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 333447168 exceeds 10% of free system memory.
2024-05-08 19:47:53.361626: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-05-08 19:47:53.489363: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-05-08 19:47:54.484191: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fcedd66db30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-08 19:47:54.484224: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Laptop GPU, 

53/53 [==============================] - 4s 22ms/step - loss: 0.5716 - accuracy: 0.6952 - val_loss: 0.4986 - val_accuracy: 0.7634
Epoch 2/100
53/53 [==============================] - 1s 12ms/step - loss: 0.5277 - accuracy: 0.7571 - val_loss: 0.5328 - val_accuracy: 0.7387
Epoch 3/100
53/53 [==============================] - 1s 12ms/step - loss: 0.4716 - accuracy: 0.7771 - val_loss: 0.4469 - val_accuracy: 0.7909
Epoch 4/100
53/53 [==============================] - 1s 12ms/step - loss: 0.4488 - accuracy: 0.8013 - val_loss: 0.4699 - val_accuracy: 0.7950
Epoch 5/100
53/53 [==============================] - 1s 11ms/step - loss: 0.4332 - accuracy: 0.8107 - val_loss: 0.4655 - val_accuracy: 0.8006
Epoch 6/100
53/53 [==============================] - 1s 11ms/step - loss: 0.3968 - accuracy: 0.8272 - val_loss: 0.4983 - val_accuracy: 0.7552
Epoch 7/100
53/53 [==============================] - 1s 11ms/step - loss: 0.3848 - accuracy: 0.8219 - val_loss: 0.4433 - val_accuracy: 0.8102
Epoch 8/100
53/53 